In [1]:
# pip install python-dotenv
# pip install psycopg2-binary
# pip install selenium
# pip install selenium-manager

In [7]:
import os
import requests
from dotenv import load_dotenv
import psycopg2
import re
import time
from db_connect import db_connect
from db import save_cafe_db,fetch_all_cafe,cafe_region_db
from congestion import compute_congestion_score



#env 파일 환경변수 로드
# load_dotenv() 

# # KaKao API 설정
# KAKAO_API_KEY = os.getenv("KAKAO_REST_API_KEY")
# headers = { "Authorization": f"KakaoAK {KAKAO_API_KEY}" }
# url = "https://dapi.kakao.com/v2/local/search/keyword.json"

In [ ]:
# 지역 별 카페 DB에 저장
cafe_region_db()

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

cur, conn = db_connect()

# 크롤링
def crawl_place_info(place_url):
    
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    try:
        driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(place_url)
        time.sleep(10)
        
        print("브라우저 실행 성공")
        wait = WebDriverWait(driver,20)
    
        blog_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'a.link_blog')))
        text = blog_element.get_attribute("textContent")
        if text:
            print("블로그 텍스트:",text.strip())
            match = re.search(r'블로그\s*(\d+)',text.strip())
            if match:
                print("크롤링 성공",match.group(1))
                return int(match.group(1))
            else:
                print("리뷰 없음")
                return 0
        else:
            print("정규식 불일치 - 리뷰 없음")
            return 0
    except Exception as e:
        print("크롤링 실패",e)
        return None
    
    finally:
        driver.quit()

cur.execute("SELECT id, place_url FROM cafe Limit 1")
rows = cur.fetchall()

for cafe_id, place_url in rows:
    count = crawl_place_info(place_url)
    if count is None:
        count = 0
    print(f"[{cafe_id}] 블로그 리뷰 수: {count}")
    
    # 블로그 리뷰 기준으로 혼잡도 계산    
    print(compute_congestion_score(count))
    
    time.sleep(1)
    
cur.close()
conn.close()

DB connect 성공
브라우저 실행 성공
블로그 텍스트: 블로그67개
크롤링 성공 67
[1] 블로그 리뷰 수: 67
4
